## Importing the LonelyBoy Library (github.com/insert-generic-name-here/lonelyboy)

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Insert-Generic-Name-Here/'))
# sys.path

In [3]:
from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp

## Importing all other Essential Libraries
#### (DO NOT FORGET TO EXECUTE THE FUNCTIONS IN THE BOTTOM CELLS)

In [4]:
import psycopg2
import numpy as np
import configparser
import pandas as pd
import geopandas as gpd
import contextily as ctx
from random import choice
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.preprocessing import MinMaxScaler
from shapely.geometry import Point, LineString, shape
from haversine import haversine

In [5]:
from multiprocessing import cpu_count, Pool
from functools import partial
import datetime

## Import Libraries for Visualizations

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')

In [8]:
from tqdm import tqdm, tqdm_notebook

## Importing the Server Credentials 

In [9]:
properties = configparser.ConfigParser()
properties.read(os.path.join('.','sql_server.ini'))
properties = properties['SERVER']

['./sql_server.ini']

## Connectiing to Server and Fetch 48hrs of Trajectory Data

In [10]:
host    = properties['host']
db_name = properties['db_name']
uname   = properties['uname']
pw      = properties['pw']
port    = properties['port']

traj_sql = 'SELECT * FROM ais_data.dynamic_ships WHERE ts>1456802710 AND ts<1456975510  '
ports_sql = 'SELECT * FROM ports.ports_of_brittany'

con = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port = port)

traj = gpd.GeoDataFrame.from_postgis(traj_sql, con, geom_col='geom' )

ports = gpd.GeoDataFrame.from_postgis(ports_sql, con, geom_col='geom' )
ports.geom = ports.geom.apply(lambda x: x[0])

print(f'Fetched {sizeof_fmt(traj.memory_usage().sum())}')
print(f'Fetched {sizeof_fmt(ports.memory_usage().sum())}')

con.close()

Fetched 38.1MiB
Fetched 14.0KiB


In [11]:
ports.head(2)
traj.head(2)

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom
0,1,port.1,1,Le Vivier-sur-Mer,35361,297025.0,2408370.0,POINT (-1.771798868659233 48.60274269672541)
1,2,port.10,10,Saint-Samson sur Rance,22327,279335.0,2396060.0,POINT (-2.001990119062326 48.48369993456267)


,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom
0,12293697,227008170,0.0,0.0,0.0,135.0,144,-4.486115,48.381535,1456802792,POINT (-4.486115 48.381535)
1,12293636,228394000,7.0,-127.0,1.7,77.0,511,-4.654577,48.123035,1456802716,POINT (-4.654577 48.123035)


## (Hopefully) Doing something Useful

* ### Select some mmsi's
* ### Denoise them (per mmsi)
* ### Resample them (per mmsi)

In [147]:
### SELECT SOME MMSI'S
# mmsis = np.array([228186700, 477115900, 227002330, 227270000, 227369960, 227298110,\
#                    228190600, 227408710, 228849000, 227730220, 228762000, 227612860,\
#                    227592820, 227590030, 227654220, 227578460, 220364000, 636092323,\
#                    227322690, 227702670, 228021700, 219118000, 227312180, 273348830,\
#                    275457000, 226084000, 244976000, 224130870, 228203800, 228167900,\
#                    227327000, 228144000, 636016457, 215477000, 226318000, 314207000,\
#                    247087700, 563187000, 477612300, 227588930, 258316000, 228919000])
mmsis = traj.mmsi.unique()

In [151]:
# mmsis = np.array([227590030, 305476000, 235005980, 226084000, 227088590])
# sample_trajectories = gspp.pick_random_group(traj, 'mmsi', group_size=50)
len(sample_trajectories)
sample_trajectories

223956

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom
0,12576916,227941000,7.0,0.0,0.0,285.0,8,-4.327213,48.100086,1456802711,POINT (-4.3272133 48.100086)
1,12576917,227705102,15.0,-127.0,0.0,261.8,511,-4.496568,48.382435,1456802711,POINT (-4.496568 48.382435)
2,12576918,227016100,0.0,0.0,0.0,264.3,174,-4.481568,48.381393,1456802713,POINT (-4.4815683 48.381393)
3,17515086,227300000,7.0,-126.0,2.8,34.2,346,-4.631805,48.111330,1456802713,POINT (-4.631805 48.11133)
4,17596828,256494000,5.0,0.0,0.0,344.0,217,-4.451149,48.383625,1456802713,POINT (-4.4511485 48.383625)
5,12576919,227008170,0.0,0.0,0.0,135.0,144,-4.486115,48.381565,1456802713,POINT (-4.486115 48.381565)
6,12576920,227574020,15.0,-127.0,0.0,241.7,511,-4.496673,48.382454,1456802713,POINT (-4.496673 48.382454)
7,12293636,228394000,7.0,-127.0,1.7,77.0,511,-4.654577,48.123035,1456802716,POINT (-4.654577 48.123035)
8,12576921,228186700,15.0,-127.0,102.3,360.0,511,-4.512498,48.370834,1456802716,POINT (-4.5124984 48.370834)
9,12293637,227006750,0.0,127.0,0.0,266.1,267,-4.484478,48.381172,1456802716,POINT (-4.4844785 48.381172)


In [152]:
### DENOISE THEM
#### DROP TIMESTAMP DUPLICATES PER MMSI
sample_trajectories = traj.loc[traj.mmsi.isin(mmsis)]
sample_trajectories = sample_trajectories.drop_duplicates(subset=['mmsi', 'ts']).sort_values('ts').reset_index(drop=True)
sample_trajectories.head()

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom
0,12576916,227941000,7.0,0.0,0.0,285.0,8,-4.327213,48.100086,1456802711,POINT (-4.3272133 48.100086)
1,12576917,227705102,15.0,-127.0,0.0,261.8,511,-4.496568,48.382435,1456802711,POINT (-4.496568 48.382435)
2,12576918,227016100,0.0,0.0,0.0,264.3,174,-4.481568,48.381393,1456802713,POINT (-4.4815683 48.381393)
3,17515086,227300000,7.0,-126.0,2.8,34.2,346,-4.631805,48.111330,1456802713,POINT (-4.631805 48.11133)
4,17596828,256494000,5.0,0.0,0.0,344.0,217,-4.451149,48.383625,1456802713,POINT (-4.4511485 48.383625)


In [153]:
### DROP OUTLIERS IN SAMPLE_TRAJECTORIES BASED ON TIMESTAMP ---- WIP; IDK IF I'LL APPLY THIS OR NOT
for mmsi in tqdm_notebook(sample_trajectories.mmsi.unique()):
    mmsi_ts_outliers = gspp.get_outliers(sample_trajectories.loc[sample_trajectories.mmsi == mmsi].ts, alpha=1.5)
#     sample_trajectories = sample_trajectories.drop(mmsi_ts_outliers)
    print (mmsi_ts_outliers)

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([117414, 117423, 117433, 117440, 117458, 117465, 117484, 117501,
            119039, 119056, 119074, 119512, 119534, 119967, 119975, 119982,
            120011, 120027, 120045, 120058, 120063, 129542, 174651, 179446,
            179467, 179505, 181826, 181859, 181882, 182176, 182185, 182206,
            182457, 183057, 183064, 183095, 183118, 183351, 183360, 183372,
            183384, 183412, 185721, 187275, 187284, 189801, 189813, 190085,
            190091, 192715, 192726, 192742, 192778, 192798, 193056, 193061,
            193079, 193089, 193344, 193397, 193692, 193697, 193718, 193726,
            193733, 193742, 193745, 193751, 194051, 194060, 194066, 194075,
            194083, 194092, 194102, 194105, 194118, 194421, 194439, 194444,
            194451, 195988, 196000, 196027, 196032, 196040, 196220, 196236,
            196243, 196270, 196277, 196502, 196513, 196557],
           dt

Int64Index([], dtype='int64')
Int64Index([212268, 212543, 218930, 219230], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')



In [156]:
### DROP OUTLIERS IN SAMPLE_TRAJECTORIES BASED ON VELOCITY (POTENTIAL-AREA-OF-ACTIVITY)
sample_trajectories['velocity'] = np.nan

for mmsi in tqdm_notebook(sample_trajectories.mmsi.unique()):
    try:
        sample_trajectories.loc[sample_trajectories.mmsi == mmsi] = gspp.calculate_velocity(sample_trajectories.loc[sample_trajectories.mmsi == mmsi], smoothing=True, window=5, center=True)
#         mmsi_vel_outliers = gspp.get_outliers(sample_trajectories.loc[sample_trajectories.mmsi == mmsi].velocity, alpha=3)
        mmsi_vel_outliers = sample_trajectories.iloc[sample_trajectories.velocity >= 102.2]
        print(mmsi_vel_outliers)
        sample_trajectories = sample_trajectories.drop(mmsi_vel_outliers)
    except:
        continue

sample_trajectories = sample_trajectories.reset_index(drop=True)
sample_trajectories = sample_trajectories.dropna(subset=['id', 'mmsi'])
sample_trajectories = sample_trajectories.fillna(0)

In [157]:
sample_trajectories

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,velocity
0,12576916.0,227941000.0,7.0,0.0,0.0,285.0,8.0,-4.327213,48.100086,1.456803e+09,POINT (-4.3272133 48.100086),0.000000
1,12576917.0,227705102.0,15.0,-127.0,0.0,261.8,511.0,-4.496568,48.382435,1.456803e+09,POINT (-4.496568 48.382435),0.000000
2,12576918.0,227016100.0,0.0,0.0,0.0,264.3,174.0,-4.481568,48.381393,1.456803e+09,POINT (-4.4815683 48.381393),0.000000
3,17515086.0,227300000.0,7.0,-126.0,2.8,34.2,346.0,-4.631805,48.111330,1.456803e+09,POINT (-4.631805 48.11133),0.000000
4,17596828.0,256494000.0,5.0,0.0,0.0,344.0,217.0,-4.451149,48.383625,1.456803e+09,POINT (-4.4511485 48.383625),0.000000
5,12576919.0,227008170.0,0.0,0.0,0.0,135.0,144.0,-4.486115,48.381565,1.456803e+09,POINT (-4.486115 48.381565),0.000000
6,12576920.0,227574020.0,15.0,-127.0,0.0,241.7,511.0,-4.496673,48.382454,1.456803e+09,POINT (-4.496673 48.382454),0.000000
7,12293636.0,228394000.0,7.0,-127.0,1.7,77.0,511.0,-4.654577,48.123035,1.456803e+09,POINT (-4.654577 48.123035),0.000000
8,12576921.0,228186700.0,15.0,-127.0,102.3,360.0,511.0,-4.512498,48.370834,1.456803e+09,POINT (-4.5124984 48.370834),0.000000
9,12293637.0,227006750.0,0.0,127.0,0.0,266.1,267.0,-4.484478,48.381172,1.456803e+09,POINT (-4.4844785 48.381172),0.000000


In [ ]:
plot_idx = 0
for mmsi in sample_trajectories.mmsi.unique():
    plt.figure(plot_idx)
    pois = gspp.get_outliers(sample_trajectories.loc[sample_trajectories.mmsi == mmsi].velocity, alpha=2)
    
    print (pois)
    sample_trajectories.loc[sample_trajectories.mmsi == mmsi].velocity.plot(figsize=(20,10), c=(0,0,0))
    for poi in pois:
        plt.axvline(x=poi, c='r')
    
    plt.show()
    plot_idx += 1

In [158]:
mmsi_resampled = []
for mmsi in tqdm_notebook(sample_trajectories.mmsi.unique()):
    tmp = gspp.resample_geospatial(sample_trajectories.loc[sample_trajectories.mmsi == mmsi], rule = '60S', method='linear', crs = {'init': 'epsg:4326'}, drop_lon_lat = True)
    tmp = tmp.drop(['id', 'status', 'turn', 'speed', 'ts'], axis=1)
    mmsi_resampled.append(tmp)
        
sample_trajectories_resampled = pd.concat(mmsi_resampled)
sample_trajectories_resampled = sample_trajectories_resampled.sort_values('datetime').reset_index(drop=True)

/home/andretri7/Documents/Insert-Generic-Name-Here/lonelyboy/geospatial/preprocessing.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sample_ves['datetime'] = pd.to_datetime(sample_ves['ts'], unit='s')


In [ ]:
plot_idx = 0
for mmsi in sample_trajectories_resampled.mmsi.unique():
    plt.figure(plot_idx)
    sample_trajectories_resampled.loc[sample_trajectories_resampled.mmsi == mmsi].velocity.plot(figsize=(20,10), c=(0,0,0))
    plt.show()
    plot_idx += 1 

In [159]:
sample_trajectories_resampled.head(20)

,mmsi,course,heading,geom,velocity,datetime
0,227941000.0,285.0,8.0,POINT (-4.3272133 48.100086),0.000000,2016-03-01 03:25:00
1,227016100.0,264.3,174.0,POINT (-4.4815683 48.381393),0.000000,2016-03-01 03:25:00
2,227300000.0,34.2,346.0,POINT (-4.631805 48.11133),0.000000,2016-03-01 03:25:00
3,256494000.0,344.0,217.0,POINT (-4.4511485 48.383625),0.000000,2016-03-01 03:25:00
4,227008170.0,135.0,144.0,POINT (-4.486115 48.381565),0.000000,2016-03-01 03:25:00
5,227574020.0,241.7,511.0,POINT (-4.496673 48.382454),0.000000,2016-03-01 03:25:00
6,228394000.0,77.0,511.0,POINT (-4.654577 48.123035),0.000000,2016-03-01 03:25:00
7,228186700.0,360.0,511.0,POINT (-4.5124984 48.370834),0.000000,2016-03-01 03:25:00
8,227006750.0,266.1,267.0,POINT (-4.4844785 48.381172),0.000000,2016-03-01 03:25:00
9,227003050.0,298.8,150.0,POINT (-4.4857283 48.38113),0.000000,2016-03-01 03:25:00


## Plot the Preprocessed Trajectories (just to be sure)

In [160]:
ax = sample_trajectories_resampled.to_crs(epsg=3857).plot(figsize=(10, 10))
ctx.add_basemap(ax, zoom=11)
plt.show()

## Search for Flocks
* #### 1. The Clustering will take place in time slices 
* #### 2. Possible Features: (X_coord, Y_coord, course)

In [189]:
# sample_datetime = np.datetime64('2016-03-02T10:04:00.000000000') 
# sample_datetime = np.datetime64('2016-03-01T19:13:00.000000000')
sample_datetime = np.random.choice(sample_trajectories_resampled.datetime)
sample_timeFrame = sample_trajectories_resampled.loc[sample_trajectories_resampled.datetime == sample_datetime].drop_duplicates(subset=['mmsi', 'datetime'])
sample_timeFrame = sample_timeFrame.sort_values('datetime').reset_index(drop=True)
# sample_timeFrame

mean_distance_to_nearest_port(sample_timeFrame, ports)

0.11710008277562464

In [191]:
sample_timeFrame

,mmsi,course,heading,geom,velocity,datetime
0,227222000.0,167.500000,167.000000,POINT (-4.4770017 48.38212),0.000000,2016-03-01 15:58:00
1,226263000.0,110.100000,113.000000,POINT (-5.113155 48.31658),27.845807,2016-03-01 15:58:00
2,227366000.0,84.692308,87.897436,POINT (-5.12575681025641 48.114685),14.034919,2016-03-01 15:58:00
3,227162950.0,232.700000,511.000000,POINT (-4.327288 48.099003),0.018671,2016-03-01 15:58:00
4,228842000.0,59.400000,511.000000,POINT (-4.310005 47.77654),0.000000,2016-03-01 15:58:00
5,234056000.0,100.500000,100.000000,POINT (-4.430343 48.309975),3.971524,2016-03-01 15:58:00
6,228236700.0,268.300000,511.000000,POINT (-4.4516034 48.12382),11.880762,2016-03-01 15:58:00
7,249297000.0,248.333333,215.000000,POINT (-4.454445 48.38507533333333),0.056727,2016-03-01 15:58:00
8,244925000.0,301.000000,301.666667,POINT (-5.102720533333333 48.32983333333333),11.220433,2016-03-01 15:58:00
9,227941000.0,124.558292,13.143966,POINT (-4.326466453775582 48.09941893154551),0.084090,2016-03-01 15:58:00


In [192]:
ax = sample_timeFrame.to_crs(epsg=3857).plot(figsize=(10, 10))
ctx.add_basemap(ax, zoom=11)
plt.show()

* ### K-Means Clustering 

In [193]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

* ##### Algorithm Prototyping...

In [ ]:
# k means determine optimal k
distortions = []
kmeans_cost = []
K = [1, 2, 3]
for k in K:
    kmeanTest = KMeans(n_clusters=k, n_init=10, n_jobs=-1, precompute_distances=True, random_state=0, verbose=0).fit(X);
    distortions.append(sum(np.min(cdist(X, kmeanTest.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
    kmeans_cost.append(kmeanTest.inertia_)
 
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(K, distortions, 'gx-')
ax2.plot(K, kmeans_cost[::-1], 'bx-')

ax1.set_xlabel('X data')
ax1.set_ylabel('Distortion', color='g')
ax2.set_ylabel('Inertia', color='b')

plt.show()

* ##### Alpha Build...

In [194]:
def k_means_elbow_criterion(X, n_sims, n_init=10, n_jobs=-1, precompute_distances=True, random_state=0, verbose=0):
    # k means determine optimal k
    distortions = []
    kmeans_cost = []
    clusters = []   
    
    for k in tqdm_notebook(range(1,n_sims+1), leave=True):
        kmeanTest = KMeans(n_clusters=k, n_init=n_init, n_jobs=n_jobs, precompute_distances=precompute_distances, random_state=random_state, verbose=verbose).fit(X);
        distortions.append(sum(np.min(cdist(X, kmeanTest.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
        kmeans_cost.append(kmeanTest.inertia_)
        clusters.append(kmeanTest.labels_)

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(list(range(1,n_sims+1)), distortions, 'gx-')
    ax2.plot(list(range(1,n_sims+1)), kmeans_cost[::-1], 'bx-')

    ax1.set_xlabel('X data')
    ax1.set_ylabel('Distortion', color='g')
    ax2.set_ylabel('Inertia', color='b')

    plt.show()    
    
    n_clusters_opt = np.argmin(np.absolute(np.array(distortions) - np.array(kmeans_cost[::-1])))
    return n_clusters_opt, clusters[n_clusters_opt]

* ##### Beta Build...

In [225]:
from sklearn.metrics import silhouette_score

def k_means_elbow_criterion(X, n_sims, n_init=10, n_jobs=-1, precompute_distances=True, random_state=0, verbose=0):
    # k means determine optimal k
    clusters = []   
    silhouettes = []
    
    for k in tqdm_notebook(range(1,n_sims+1), leave=True):
        kmeanTest = KMeans(n_clusters=k, n_init=n_init, n_jobs=n_jobs, precompute_distances=precompute_distances, random_state=random_state, verbose=verbose).fit(X);
        label = kmeanTest.labels_
        
        clusters.append(kmeanTest.labels_)

        try:
            sil_coeff = silhouette_score(X, label, metric='euclidean')
            silhouettes.append(sil_coeff)
#             print(f"For n_clusters={k}, The Silhouette Coefficient is {sil_coeff}")
        except ValueError:
            silhouettes.append(0)
        
#     pd.DataFrame(silhouettes).plot()
    n_clusters_opt = np.argmax(np.array(silhouettes))
    return n_clusters_opt+1, clusters[n_clusters_opt]

In [224]:
from sklearn.preprocessing import StandardScaler


def flock_discovery(gdf, n_init=10, n_jobs=-1, precompute_distances=True, random_state=0, verbose=0):
    gdf[['lon', 'lat']] = gdf['geom'].apply(lambda x: pd.Series({'lon':x.x, 'lat':x.y})) 
    gdf = gdf.drop('geom', axis=1) 
    
    datetimes = gdf['datetime'].unique()
    flocks = {}
    
    for datetime_of_interest in tqdm_notebook(datetimes):
#         timeFrame = gdf[['lon','lat', 'course']].loc[gdf['datetime'] == datetime_of_interest]
        timeFrame = gdf[['lon','lat']].loc[gdf['datetime'] == datetime_of_interest]
        
        scaler = MinMaxScaler()
        X_std = scaler.fit_transform(timeFrame.values)

        if (len(timeFrame) == 1):
            continue
        n_flocks, labels = k_means_elbow_criterion(X_std, len(timeFrame), n_init, n_jobs, precompute_distances, random_state, verbose)
        flocks[str(datetime_of_interest)] = (n_flocks, timeFrame.index, labels)
    
    return flocks

In [226]:
# gdf_tmp = sample_trajectories_resampled.loc[sample_trajectories_resampled.datetime == sample_datetime].drop_duplicates(subset=['mmsi', 'datetime'])  
flocks = flock_discovery(sample_timeFrame)

In [227]:
flocks

{'2016-03-01T15:58:00.000000000': (11,
  Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
              17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
              34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
              51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64],
             dtype='int64'),
  array([ 0, 10,  6,  2,  4,  0,  2,  0, 10,  2,  8,  4,  6,  2,  0,  0,  0,
          0,  6,  7,  0,  5,  7,  7,  4,  0,  9,  1,  0,  6,  0,  0,  0,  0,
          0,  1,  0,  0,  0,  0,  4,  2, 10,  0,  0,  0,  7,  7,  7,  2,  6,
          0,  0,  0,  0,  0,  9,  0,  5,  0,  0,  0,  0,  3,  0],
        dtype=int32))}

In [230]:
LABEL_COLOR_MAP = {0 : 'black',\
                   1 : 'r',\
                   2 : 'g',\
                   3 : 'b',\
                   4 : 'm',\
                   5 : 'y',\
                   6 : 'maroon',\
                   7 : 'pink',\
                   8 : 'sienna',\
                   9 : 'darkslategray',\
                   10 : 'purple'}

label_color = [LABEL_COLOR_MAP[l] for l in flocks[str(sample_timeFrame.datetime.unique()[0])][2]]

In [231]:
ax = sample_timeFrame.to_crs(epsg=3857).plot(figsize=(10, 10), c=label_color)
ctx.add_basemap(ax, zoom=11)
plt.show()

* ### Clustering using MeanShift

In [232]:
from sklearn.cluster import MeanShift

In [234]:
gdf_tmp = sample_trajectories_resampled.loc[sample_trajectories_resampled.datetime == sample_datetime]

gdf_tmp[['lon', 'lat']] = gdf_tmp['geom'].apply(lambda x: pd.Series({'lon':x.x, 'lat':x.y})) 
gdf_tmp = gdf_tmp.drop('geom', axis=1) 
gdf_tmp[['lon', 'lat', 'course']] 
# X = gdf_tmp[['lon', 'lat', 'course']].values
X = gdf_tmp[['lon', 'lat']].values
X

,lon,lat,course
47193,-4.477002,48.382120,167.500000
47194,-4.490441,48.379167,0.000000
47195,-5.453907,48.250305,221.400000
47196,-4.496584,48.382445,118.017555
47197,-4.487266,48.380714,192.711236
47198,-4.486098,48.381550,135.000000
47199,-4.487208,48.363489,329.141509
47200,-4.156922,47.830750,236.545652
47201,-4.339067,48.097646,0.000000
47202,-4.851352,48.231980,15.800000


array([[-4.4770017 , 48.38212   ],
       [-4.49044106, 48.37916667],
       [-5.4539065 , 48.250305  ],
       [-4.49658379, 48.38244511],
       [-4.4872658 , 48.38071374],
       [-4.4860983 , 48.38155   ],
       [-4.48720829, 48.36348943],
       [-4.15692208, 47.83075027],
       [-4.3390667 , 48.0976463 ],
       [-4.8513517 , 48.23198   ],
       [-4.4734388 , 48.33964233],
       [-4.511763  , 48.364624  ],
       [-4.47827933, 48.383146  ],
       [-4.990093  , 48.480907  ],
       [-4.81036463, 48.37642633],
       [-4.77959877, 48.35997324],
       [-4.3978515 , 48.188812  ],
       [-4.955488  , 48.099434  ],
       [-4.49042446, 48.35528316],
       [-4.4533315 , 48.38165   ],
       [-4.497103  , 48.380352  ],
       [-4.49546213, 48.38341502],
       [-4.4279466 , 48.30187   ],
       [-4.5565333 , 47.994648  ],
       [-4.48828096, 48.37841056],
       [-5.62337601, 48.54783519],
       [-4.48488944, 48.37891111],
       [-4.48844125, 48.37731   ],
       [-4.49829497,

In [235]:
scaler = MinMaxScaler()
X_std = scaler.fit_transform(X)

clustering = MeanShift(cluster_all=False, n_jobs=-1).fit(X_std)
clustering.labels_

,0,1
count,65.000000,65.000000
mean,0.795002,0.654686
std,0.180474,0.233828
min,0.000000,0.000000
25%,0.747563,0.492681
50%,0.871162,0.784886
75%,0.875854,0.797369
max,1.000000,1.000000


array([0, 0, 2, 0, 0, 0, 0, 3, 1, 2, 0, 0, 0, 4, 0, 0, 1, 2, 0, 0, 0, 0,
       0, 1, 0, 4, 0, 0, 0, 0, 0, 6, 0, 0, 2, 2, 1, 3, 0, 1, 0, 2, 1, 5,
       3, 2, 1, 0, 0, 0, 0, 2, 0, 0, 4, 0, 0, 3, 0, 1, 2, 0, 1, 0, 0])

In [236]:
LABEL_COLOR_MAP = {-1: 'black',\
                   0 : 'r',\
                   1 : 'g',\
                   2 : 'b',\
                   3 : 'y',\
                   4 : 'm',\
                   5 : 'maroon',\
                   6 : 'pink',\
                   7 : 'sienna',\
                   8 : 'darkslategray',\
                   9 : 'purple'}

label_color = [LABEL_COLOR_MAP[l] for l in clustering.labels_]

In [237]:
ax = sample_timeFrame.to_crs(epsg=3857).plot(figsize=(10, 10), c=label_color)
ctx.add_basemap(ax, zoom=11)
plt.show()

* ### Clustering using DBSCAN
* ##### NOTE-TO-SELF: Due to the fact that is a Density Clustering Algorithm, it may be not a good idea... Definitely will be used in Convoy Mining

In [238]:
gdf_tmp = sample_trajectories_resampled.loc[sample_trajectories_resampled.datetime == sample_datetime]

gdf_tmp[['lon', 'lat']] = gdf_tmp['geom'].apply(lambda x: pd.Series({'lon':x.x, 'lat':x.y})) 
gdf_tmp = gdf_tmp.drop('geom', axis=1) 
gdf_tmp[['lon', 'lat', 'course']] 
# X = gdf_tmp[['lon', 'lat', 'course']].values
X = gdf_tmp[['lon', 'lat']].values
X

/home/andretri7/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,lon,lat,course
47193,-4.477002,48.382120,167.500000
47194,-4.490441,48.379167,0.000000
47195,-5.453907,48.250305,221.400000
47196,-4.496584,48.382445,118.017555
47197,-4.487266,48.380714,192.711236
47198,-4.486098,48.381550,135.000000
47199,-4.487208,48.363489,329.141509
47200,-4.156922,47.830750,236.545652
47201,-4.339067,48.097646,0.000000
47202,-4.851352,48.231980,15.800000


array([[-4.4770017 , 48.38212   ],
       [-4.49044106, 48.37916667],
       [-5.4539065 , 48.250305  ],
       [-4.49658379, 48.38244511],
       [-4.4872658 , 48.38071374],
       [-4.4860983 , 48.38155   ],
       [-4.48720829, 48.36348943],
       [-4.15692208, 47.83075027],
       [-4.3390667 , 48.0976463 ],
       [-4.8513517 , 48.23198   ],
       [-4.4734388 , 48.33964233],
       [-4.511763  , 48.364624  ],
       [-4.47827933, 48.383146  ],
       [-4.990093  , 48.480907  ],
       [-4.81036463, 48.37642633],
       [-4.77959877, 48.35997324],
       [-4.3978515 , 48.188812  ],
       [-4.955488  , 48.099434  ],
       [-4.49042446, 48.35528316],
       [-4.4533315 , 48.38165   ],
       [-4.497103  , 48.380352  ],
       [-4.49546213, 48.38341502],
       [-4.4279466 , 48.30187   ],
       [-4.5565333 , 47.994648  ],
       [-4.48828096, 48.37841056],
       [-5.62337601, 48.54783519],
       [-4.48488944, 48.37891111],
       [-4.48844125, 48.37731   ],
       [-4.49829497,

In [245]:
from sklearn.cluster import DBSCAN

scaler = MinMaxScaler()
X_std = scaler.fit_transform(X)

# clustering = DBSCAN(eps=3, min_samples=2).fit(X)
clustering = DBSCAN(min_samples=2).fit(X_std)
clustering.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [143]:
LABEL_COLOR_MAP = {-1: 'black',\
                   0 : 'r',\
                   1 : 'g',\
                   2 : 'b',\
                   3 : 'y',\
                   4 : 'm'}

label_color = [LABEL_COLOR_MAP[l] for l in clustering.labels_]

In [144]:
ax = sample_timeFrame.to_crs(epsg=3857).plot(figsize=(10, 10), c=label_color)
ctx.add_basemap(ax, zoom=11)
plt.show()

### Clustering using Hierarchical Clustering 
* ##### TODO: Make a function to find the optimal cluster number 

In [56]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster

In [234]:
# generate the linkage matrix
ZC = linkage(X, 'complete')

# calculate full dendrogram
plt.figure(1, figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram -- Complete-Linkage')
plt.xlabel('X[i]')
plt.ylabel('distance')
dendrogram(
    ZC,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

<Figure size 2500x1000 with 0 Axes>

Text(0.5, 1.0, 'Hierarchical Clustering Dendrogram -- Complete-Linkage')

Text(0.5, 0, 'X[i]')

Text(0, 0.5, 'distance')

{'icoord': [[15.0, 15.0, 25.0, 25.0], [5.0, 5.0, 20.0, 20.0]],
 'dcoord': [[0.0, 5.189871935559221, 5.189871935559221, 0.0],
  [0.0, 73.5846642690964, 73.5846642690964, 5.189871935559221]],
 'ivl': ['2', '0', '1'],
 'leaves': [2, 0, 1],
 'color_list': ['g', 'b']}

## Search for Convoys

# FUNCTIONS

In [1]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [110]:
def make_lines(gdf, df_out, i, geometry = 'geometry'):
    geom0 = gdf.loc[i][geometry]
    geom1 = gdf.loc[i + 1][geometry]
    
    start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
    line = LineString([start, end])
    
    # Create a DataFrame to hold record
    data = {'id': i,
            'geometry': [line]}
    df_line = pd.DataFrame(data, columns = ['id', 'geometry'])
    
    # Add record DataFrame of compiled records
    df_out = pd.concat([df_out, df_line])
    return df_out

In [170]:
def mean_distance_to_nearest_port(gdf, ports):
    '''
    Calculates the minimum distance between the point and the lists of ports. Can be used to determine if the ship is sailing or not
    '''
    counter = 0
    for point in tqdm_notebook(gdf.geom):
        counter += ports.geom.distance(point).min()
    
    return counter/len(gdf)